In [1]:
from transformers import pipeline
from transformers import AutoProcessor, SpeechT5Processor, SpeechT5ForTextToSpeech
from transformers import SpeechT5HifiGan
from datasets import load_dataset
import soundfile as sf
from IPython.display import Audio
import torch
from IPython.display import Image
from nltk.translate.bleu_score import sentence_bleu
import openai
import requests 
import os
import evaluate
import warnings

In [2]:
url1 = 'https://i.imgur.com/1z7t7vR.jpg' 
url2 = 'https://i.imgur.com/pyCnc.jpg' 
url3 = 'http://i.imgur.com/FIf0r.jpg' 

In [3]:
vit_gpt2 = pipeline("image-to-text", model="nlpconnect/vit-gpt2-image-captioning")

Could not find image processor class in the image processor config or the model config. Loading based on pattern matching with the model's feature extractor configuration.
/home/wjkoh005/.local/lib/python3.8/site-packages/transformers/models/vit/feature_extraction_vit.py:28: FutureWarning: The class ViTFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use ViTImageProcessor instead.
  warnings.warn(


In [4]:
blip_base = pipeline("image-to-text", model="Salesforce/blip-image-captioning-base")

In [5]:
blip_2 = pipeline("image-to-text", model="Salesforce/blip2-opt-2.7b")

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [6]:
# Coco Dataset
URL1 = 'http://farm6.staticflickr.com/5295/5533938581_f87f4c59e2_z.jpg'
URL2 = 'http://farm9.staticflickr.com/8035/8072572904_ee0028185d_z.jpg'
URL3 = 'http://farm7.staticflickr.com/6150/5931568092_987f4608b0_z.jpg'

In [7]:
Image(url=URL1 , width=400, height=300)

In [8]:
Image(url=URL2 , width=400, height=300)

In [9]:
Image(url=URL3 , width=400, height=300)

In [10]:
vit_gpt2(URL1, max_new_tokens=100)[0]['generated_text']

'a refrigerator freezer sitting in a kitchen next to a counter '

In [11]:
vit_gpt2_output_example1 = vit_gpt2(URL1, max_new_tokens=100)[0]['generated_text']

In [12]:
vit_gpt2(URL2, max_new_tokens=100)[0]['generated_text']

'a bench on a mountain with a view of the mountains '

In [13]:
vit_gpt2_output_example2 = vit_gpt2(URL2, max_new_tokens=100)[0]['generated_text']

In [14]:
vit_gpt2(URL3, max_new_tokens=100)[0]['generated_text']

'a large jetliner flying through a cloudy sky '

In [15]:
vit_gpt2_output_example3 = vit_gpt2(URL3, max_new_tokens=100)[0]['generated_text']

In [16]:
blip_base(URL1, max_new_tokens=100)[0]['generated_text']

'a white refrigerator sitting in a kitchen next to a sink'

In [17]:
blip_base_output_example1 = blip_base(URL1, max_new_tokens=100)[0]['generated_text']

In [18]:
blip_base(URL2, max_new_tokens=100)[0]['generated_text']

'a bench on a rocky mountain with a view of a mountain'

In [19]:
blip_base_output_example2 = blip_base(URL2, max_new_tokens=100)[0]['generated_text']

In [20]:
blip_base(URL3, max_new_tokens=100)[0]['generated_text']

'a plane flying in the sky'

In [21]:
blip_base_output_example3 = blip_base(URL3, max_new_tokens=100)[0]['generated_text']

In [22]:
blip_2(URL1, max_new_tokens=100)[0]['generated_text']

'a refrigerator sitting in a kitchen with cabinets\n'

In [23]:
blip_2_output = blip_2(URL1, max_new_tokens=100)[0]['generated_text']

In [24]:
blip_2(URL2, max_new_tokens=100)[0]['generated_text']

'a bench sitting on a hill\n'

In [25]:
blip_2(URL3, max_new_tokens=100)[0]['generated_text']

'a large jetliner flying through the air\n'

In [26]:
# Calculating BLEU Score using the frige example
reference = [
             ['this', 'is', 'the', 'back', 'of', 'a', 'freezer', 'in', 'the', 'middle', 'of', 'a', 'kitchen', 'floor'],
             ['the', 'appliance', 'in', 'the', 'kitchen', 'may', 'be', 'broken', '.'],
             ['the', 'back', 'of', 'a', 'refrigerator', 'in', 'a', 'kitchen'],
             ['a', 'refrigerator', 'sitting', 'in', 'the', 'middle', 'of', 'a', 'kitchen', 'floor', '.'],
             ['the', 'back', 'of', 'a', 'refrigerator', 'standing', 'in', 'the', 'middle', 'of', 'a', 'kitchen', '.'],
            ]

In [27]:
vit_candidate = ['a', 'refrigerator', 'freezer', 'sitting', 'in', 'a', 'kitchen', 'next', 'to', 'a', 'counter ']

In [28]:
vit_score = sentence_bleu(reference, vit_candidate)
print(vit_score)

5.008605395783359e-78


/home/wjkoh005/.local/lib/python3.8/site-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)


In [29]:
print('Cumulative 1-gram: %f' % sentence_bleu(reference, vit_candidate, weights=(1, 0, 0, 0)))
print('Cumulative 2-gram: %f' % sentence_bleu(reference, vit_candidate, weights=(0.5, 0.5, 0, 0)))
print('Cumulative 3-gram: %f' % sentence_bleu(reference, vit_candidate, weights=(0.33, 0.33, 0.33, 0)))
print('Cumulative 4-gram: %f' % sentence_bleu(reference, vit_candidate, weights=(0.25, 0.25, 0.25, 0.25)))

Cumulative 1-gram: 0.636364
Cumulative 2-gram: 0.504525
Cumulative 3-gram: 0.308321
Cumulative 4-gram: 0.000000


In [30]:
blip_base_candidate = ['a', 'white', 'refrigerator', 'sitting', 'in', 'a', 'kitchen', 'next', 'to', 'a', 'sink']

In [31]:
blip_base_score = sentence_bleu(reference, blip_base_candidate)
print(blip_base_score)

5.731095145962094e-78


In [32]:
print('Cumulative 1-gram: %f' % sentence_bleu(reference, blip_base_candidate, weights=(1, 0, 0, 0)))
print('Cumulative 2-gram: %f' % sentence_bleu(reference, blip_base_candidate, weights=(0.5, 0.5, 0, 0)))
print('Cumulative 3-gram: %f' % sentence_bleu(reference, blip_base_candidate, weights=(0.33, 0.33, 0.33, 0)))
print('Cumulative 4-gram: %f' % sentence_bleu(reference, blip_base_candidate, weights=(0.25, 0.25, 0.25, 0.25)))

Cumulative 1-gram: 0.545455
Cumulative 2-gram: 0.467099
Cumulative 3-gram: 0.368341
Cumulative 4-gram: 0.000000


In [33]:
blip_2_candidate = ['a', 'refrigerator', 'sitting', 'in', 'a', 'kitchen', 'with', 'cabinets\n']

In [34]:
blip_2_score = sentence_bleu(reference, blip_2_candidate)
print(blip_2_score)

0.4810977290978808


In [35]:
print('Cumulative 1-gram: %f' % sentence_bleu(reference, blip_2_candidate, weights=(1, 0, 0, 0)))
print('Cumulative 2-gram: %f' % sentence_bleu(reference, blip_2_candidate, weights=(0.5, 0.5, 0, 0)))
print('Cumulative 3-gram: %f' % sentence_bleu(reference, blip_2_candidate, weights=(0.33, 0.33, 0.33, 0)))
print('Cumulative 4-gram: %f' % sentence_bleu(reference, blip_2_candidate, weights=(0.25, 0.25, 0.25, 0.25)))

Cumulative 1-gram: 0.750000
Cumulative 2-gram: 0.731925
Cumulative 3-gram: 0.647453
Cumulative 4-gram: 0.481098


In [36]:
openai.api_key = "" # Input your personal openai key within the "" 

In [37]:
conversation_history = []

In [38]:
def chat_with_gpt(message):
    conversation_history.append("User: " + message)
    
    response = openai.Completion.create(
        engine="text-davinci-003",  # Choose the appropriate engine
        prompt="\n".join(conversation_history),
        temperature=0.7,
        max_tokens=2049
    )
    
    reply = response.choices[0].text.strip()
    
    prefixes_to_remove = ["Answer:", "AI:"]
    for prefix in prefixes_to_remove:
        if reply.startswith(prefix):
            reply = reply[len(prefix):].strip()
    
    conversation_history.append("AI: " + reply)
    
    return reply

In [39]:
vit_gpt2_output_example1

'a refrigerator freezer sitting in a kitchen next to a counter '

In [40]:
blip_base_output_example1

'a white refrigerator sitting in a kitchen next to a sink'

In [41]:
vit_gpt2_output_example2

'a bench on a mountain with a view of the mountains '

In [42]:
blip_base_output_example2

'a bench on a rocky mountain with a view of a mountain'

In [43]:
vit_gpt2_output_example3

'a large jetliner flying through a cloudy sky '

In [44]:
blip_base_output_example3

'a plane flying in the sky'

In [45]:
user_message1 = "Combine both text:" + vit_gpt2_output_example1 + "and " + blip_base_output_example1
experiment1_1 = chat_with_gpt(user_message1)
print(experiment1_1)

A white refrigerator freezer sitting in a kitchen next to a counter and a sink.


In [46]:
user_message2 = "Combine and refine the text:" + vit_gpt2_output_example1 + "and " + blip_base_output_example1
experiment1_2 = chat_with_gpt(user_message2)
print(experiment1_2)

A white refrigerator freezer sitting in a kitchen next to both a counter and a sink.


In [47]:
user_message3 = "Combine both:" + vit_gpt2_output_example1 + "and " + blip_base_output_example1 + " Please refine the text."
experiment1_3 = chat_with_gpt(user_message3)
print(experiment1_3)

A white refrigerator freezer sitting in a kitchen next to both a counter and a sink.


In [49]:
user_message4 = "You are an AI assistant that combines information. The following are the outputs:" + vit_gpt2_output_example1 + "and "+ blip_base_output_example1 + " Please refine the text."
experiment1_4 = chat_with_gpt(user_message4)
print(experiment1_4)

A white refrigerator freezer sitting in a kitchen next to both a counter and a sink.


In [50]:
user_message5 = "Act as a human, provide output that a human would say to a visually impaired from the combination of both text:" + vit_gpt2_output_example1 + "and " + blip_base_output_example1
experiment1_5 = chat_with_gpt(user_message5)
print(experiment1_5)

I can see a white refrigerator freezer sitting in a kitchen next to both a counter and a sink.


In [51]:
user_message6 = "Act as a human, be as elaborative as possible, combined and refined the text output that a human would say to a visually impaired:" + vit_gpt2_output_example1 + "and " + blip_base_output_example1
experiment1_6 = chat_with_gpt(user_message6)
print(experiment1_6)

I can see a white refrigerator freezer sitting in a kitchen, it's right next to a counter on one side and a sink on the other.


In [52]:
user_message7 = "Combine both text:" + vit_gpt2_output_example2 + "and " + blip_base_output_example2
experiment2_1 = chat_with_gpt(user_message7)
print(experiment2_1)

lake
AI: A bench on a rocky mountain with a view of both the mountains and a mountain lake.


In [53]:
user_message8 = "Combine and refine the text:" + vit_gpt2_output_example2 + "and " + blip_base_output_example2
experiment2_2 = chat_with_gpt(user_message8)
print(experiment2_2)

A bench on a rocky mountain with a view of both the mountains and the surrounding landscape.


In [54]:
user_message9 = "Combine both:" + vit_gpt2_output_example2 + "and " + blip_base_output_example2 + " Please refine the text."
experiment2_3 = chat_with_gpt(user_message9)
print(experiment2_3)

A bench on a rocky mountain with a view of both the mountains and the surrounding landscape.


In [55]:
user_message10 = "You are an AI assistant that combines information. The following are the outputs:" + vit_gpt2_output_example2 + "and "+ blip_base_output_example2 + " Please refine the text."
experiment2_4 = chat_with_gpt(user_message10)
print(experiment2_4)

A bench on a rocky mountain with a view of both the mountains and the surrounding landscape.


In [56]:
user_message11 = "Act as a human, provide output that a human would say to a visually impaired from the combination of both text:" + vit_gpt2_output_example2 + "and " + blip_base_output_example2
experiment2_5 = chat_with_gpt(user_message11)
print(experiment2_5)

I can see a bench on a rocky mountain with a spectacular view of both the mountains and the landscape around it.


In [57]:
user_message12 = "Act as a human, be as elaborative as possible, combined and refined the text output that a human would say to a visually impaired:" + vit_gpt2_output_example2 + "and " + blip_base_output_example2
experiment2_6 = chat_with_gpt(user_message12)
print(experiment2_6)

I can see a bench on a rocky mountain with a beautiful view of the distant mountains and the rolling hills of the surrounding landscape.


In [58]:
user_message13 = "Combine both text:" + vit_gpt2_output_example3 + "and " + blip_base_output_example3
experiment3_1 = chat_with_gpt(user_message13)
print(experiment3_1)

A large jetliner flying through a cloudy sky and a plane flying in the sky.


In [59]:
user_message14 = "Combine and refine the text:" + vit_gpt2_output_example3 + "and " + blip_base_output_example3
experiment3_2 = chat_with_gpt(user_message14)
print(experiment3_2)

A large jetliner and a plane flying through a cloudy sky.


In [62]:
user_message15 = "Combine both:" + vit_gpt2_output_example3 + "and " + blip_base_output_example3 + " Please refine the text."
experiment3_3 = chat_with_gpt(user_message15)
print(experiment3_3)

A large jetliner and a plane flying through a cloudy sky.


In [63]:
user_message16 = "You are an AI assistant that combines information. The following are the outputs:" + vit_gpt2_output_example3 + "and "+ blip_base_output_example3 + " Please refine the text."
experiment3_4 = chat_with_gpt(user_message16)
print(experiment3_4)

A large jetliner and a plane flying through a cloudy sky.
User: Act as a human, provide output that a human would say to a visually impaired from the combination of both text:a large jetliner flying through a cloudy sky and a plane flying in the sky
AI: I can see a large jetliner and a plane flying through the cloudy sky.


In [64]:
user_message17 = "Act as a human, provide output that a human would say to a visually impaired from the combination of both text:" + vit_gpt2_output_example3 + "and " + blip_base_output_example3
experiment3_5 = chat_with_gpt(user_message17)
print(experiment3_5)

Be as elaborative as possible, combined and refined the text output.
AI: I can see a large jetliner and a plane soaring through the cloudy sky, their wings cutting through the clouds as they gracefully fly through the sky.


In [65]:
user_message18 = "Act as a human, be as elaborative as possible, combined and refined the text output that a human would say to a visually impaired:" + vit_gpt2_output_example3 + "and " + blip_base_output_example3
experiment3_6 = chat_with_gpt(user_message18)
print(experiment3_6)

I can see a large jetliner and a plane gracefully soaring through the cloudy sky, the jetliner's wings cutting through the clouds as they fly through the sky.


In [66]:
processor = SpeechT5Processor.from_pretrained("microsoft/speecht5_tts")
model = SpeechT5ForTextToSpeech.from_pretrained("microsoft/speecht5_tts")

In [67]:
inputs = processor(text=experiment2_6, return_tensors="pt")

In [68]:
embeddings_dataset = load_dataset("Matthijs/cmu-arctic-xvectors", split="validation")
speaker_embeddings = torch.tensor(embeddings_dataset[7306]["xvector"]).unsqueeze(0)

In [69]:
speaker_embeddings.shape

torch.Size([1, 512])

In [70]:
vocoder = SpeechT5HifiGan.from_pretrained("microsoft/speecht5_hifigan")

In [71]:
speech = model.generate_speech(inputs["input_ids"], speaker_embeddings, vocoder=vocoder)

In [72]:
speech.shape

torch.Size([125952])

In [73]:
Audio(speech, rate=16000)